In [61]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pandas as pd

## Load, Aggregate, and Shape Data into Train and Test Sets

In [62]:
### CURRENTLY ONLY FOR THE 2018 SEASON !!! EASY TO EXPAND ###

target = pd.read_csv("Y_data.csv")
input = pd.read_csv("X_2018-2019_final.csv")

# Match input data to output data by date.
data = pd.merge(input, target, how='inner')
data.fillna(0)

# Separate back into Y and X data.
Y_data = data.iloc[:, 53:67]
X_data = data.iloc[:, 1:53]

In [63]:
Y_data.head()

,size,likelihood,type,atl,ntl,btl,N,NE,E,SE,S,SW,W,NW
0,2,3,1,3,3,2,1,1,1,1,0,0,0,1
1,1,1,1,1,1,0,1,1,1,1,0,0,0,1
2,1,2,1,2,2,2,1,1,1,1,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,0,0,0,1
4,1,1,5,1,1,1,1,1,1,1,0,0,0,1


In [64]:
X_data.head()

,weasd,air,albedo,lftx4,cfrzr,cicep,crain,csnow,cdcon,acpcp,...,vstm,vwnd,wcinc,wcconv,wcvflx,wcuflx,wvinc,wvconv,wvvflx,wvuflx
0,3.462348,275.196591,20.293056,1.818175,0.0,0.0,0.263889,0.541667,34.763889,1.502604,...,6.230269,4.795019,0.368141,0.426346,82339.744667,73290.042889,1.154290,0.715487,1.047864e+06,719503.900000
1,15.962565,273.186932,35.004167,3.374641,0.0,0.0,0.194444,0.486111,12.958333,0.059028,...,-10.865753,0.306765,0.075161,-0.135596,-17681.848667,48968.488889,0.142669,-0.465670,-2.757419e+05,766487.877778
2,18.088976,270.526612,35.952778,3.140869,0.0,0.0,0.083333,0.375000,32.638889,1.087674,...,-11.011229,2.793661,-0.033226,0.275993,1473.177264,62574.111667,0.013199,0.921517,-4.299622e+03,926458.622222
3,22.829102,268.677762,35.254167,5.315925,0.0,0.0,0.013889,0.305556,14.583333,0.326389,...,-19.357874,-0.505963,0.065596,-0.183372,-21280.395333,30643.284444,0.501735,-1.024022,-3.478883e+05,423369.244444
4,22.464518,267.055167,35.647223,9.787543,0.0,0.0,0.000000,0.069444,0.000000,0.000000,...,-15.446493,1.451025,-0.000070,0.032900,-13023.525778,35952.103111,0.003099,-0.292874,-2.029971e+05,500196.133333


In [65]:
# Copy the data over to new df.
X_scaled = X_data.copy()
X_scaled = X_scaled.fillna(0)

# Normalize our features.
for column in X_scaled.columns:
    X_scaled[column] = (X_scaled[column] - 
                        X_scaled[column].mean()) / X_scaled[column].std()    

X_scaled = X_scaled.fillna(0)
X_scaled.head()

,weasd,air,albedo,lftx4,cfrzr,cicep,crain,csnow,cdcon,acpcp,...,vstm,vwnd,wcinc,wcconv,wcvflx,wcuflx,wvinc,wvconv,wvvflx,wvuflx
0,-1.112741,0.431778,-1.048367,-1.385287,0.0,-0.178981,1.475401,1.333716,1.734011,1.477702,...,0.874114,0.826359,1.370362,1.129613,1.540871,0.935115,1.362094,1.075823,1.105632,0.264083
1,-0.419734,-0.082726,0.933026,-0.582834,0.0,-0.178981,0.924992,1.113469,0.290258,-0.363503,...,-0.622979,-0.634655,-0.085734,-1.230877,-0.900512,0.305563,-0.230936,-0.798989,-0.817469,0.351993
2,-0.301847,-0.763809,1.060791,-0.703358,0.0,-0.178981,0.044338,0.672976,1.593314,0.948481,...,-0.635718,0.174878,-0.624414,0.498041,-0.432965,0.657738,-0.434816,1.402847,-0.423083,0.651311
3,-0.039056,-1.237143,0.966698,0.418016,0.0,-0.178981,-0.506070,0.397668,0.397849,-0.022498,...,-1.366631,-0.899214,-0.133273,-1.431566,-0.988347,-0.168776,0.334497,-1.685243,-0.922292,-0.290008
4,-0.059268,-1.652552,1.019637,2.723406,0.0,-0.178981,-0.616152,-0.538381,-0.567718,-0.438789,...,-1.024113,-0.262176,-0.459630,-0.523094,-0.786809,-0.031360,-0.450721,-0.524715,-0.711776,-0.146259


In [66]:
# Split data into training and testing subsets.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_data, test_size=0.30)

## Generate SVM OneVsRest Models

### Size

In [67]:
# Isolate size feature and generate model.
y_size = y_train['size']
size_model = svm.SVC()
size_ovr = OneVsRestClassifier(size_model).fit(X_train, y_size)
size_preds = size_ovr.predict(X_test)
size_preds

array([2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [68]:
# Calculate accuracy with y_test.
accuracy_score(y_test['size'], size_preds)


0.5909090909090909

### Likelihood

In [69]:
# Isolate likelihood feature and generate model.
y_likelihood = y_train['likelihood']
likelihood_model = svm.SVC()
likelihood_ovr = OneVsRestClassifier(likelihood_model).fit(X_train, y_likelihood)
likelihood_preds = likelihood_ovr.predict(X_test)
likelihood_preds

array([2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [70]:
# Calculate accuracy with y_test.
accuracy_score(y_test['likelihood'], likelihood_preds)

0.5227272727272727

### Problem Type

In [71]:
# Isolate problem type feature and generate model.
y_type = y_train['type']
type_model = svm.SVC()
type_ovr = OneVsRestClassifier(type_model).fit(X_train, y_type)
type_preds = type_ovr.predict(X_test)
type_preds

array([1, 1, 1, 1, 3, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3,
       1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 3, 1, 1, 1])

In [72]:
# Calculate accuracy with y_test.
accuracy_score(y_test['type'], type_preds)

0.8181818181818182

### Above Treeline Risk

In [73]:
# Isolate atl feature and generate model.
y_atl = y_train['atl']
atl_model = svm.SVC()
atl_ovr = OneVsRestClassifier(atl_model).fit(X_train, y_atl)
atl_preds = atl_ovr.predict(X_test)
atl_preds

array([2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1])

In [74]:
# Calculate accuracy with y_test.
accuracy_score(y_test['atl'], atl_preds)

0.5909090909090909

### Near Treeline Risk

In [75]:
# Isolate ntl feature and generate model.
y_ntl = y_train['ntl']
ntl_model = svm.SVC()
ntl_ovr = OneVsRestClassifier(ntl_model).fit(X_train, y_ntl)
ntl_preds = ntl_ovr.predict(X_test)
ntl_preds

array([2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1])

In [76]:
# Calculate accuracy with y_test.
accuracy_score(y_test['ntl'], ntl_preds)

0.5909090909090909

### Below Treeline Risk

In [77]:
# Isolate atl feature and generate model.
y_btl = y_train['btl']
btl_model = svm.SVC()
btl_ovr = OneVsRestClassifier(btl_model).fit(X_train, y_btl)
btl_preds = btl_ovr.predict(X_test)
btl_preds

array([1, 1, 0, 2, 0, 3, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 3, 1, 0, 1, 0, 3, 1, 3])

In [78]:
# Calculate accuracy with y_test.
accuracy_score(y_test['btl'], btl_preds)

0.6818181818181818

### Aspect

In [79]:
y_train.head()

,size,likelihood,type,atl,ntl,btl,N,NE,E,SE,S,SW,W,NW
96,1,1,1,1,1,1,1,1,1,1,0,0,1,1
58,1,1,1,1,1,0,1,1,1,1,0,0,0,1
49,1,1,1,1,1,0,1,1,1,1,1,1,1,1
56,0,0,0,0,0,0,1,1,1,1,1,1,1,1
77,2,2,1,2,2,1,1,1,1,1,1,0,0,1


In [80]:
aspect = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

In [81]:
# Format test set.
y_train_asp = y_train.iloc[:, 6:14]
y_test_asp = y_test.iloc[:, 6:14]

for feature in aspect:
    model = svm.SVC()
    model.fit(X_train, y_train_asp[feature])
    preds = model.predict(X_test)
    score = accuracy_score(y_test[feature], preds)
    print(feature + ": ", score)

N:  0.9318181818181818
NE:  0.9318181818181818
E:  0.9772727272727273
SE:  0.9545454545454546
S:  0.7954545454545454
SW:  0.7954545454545454
W:  0.7727272727272727
NW:  0.9545454545454546
